In this Jupyter Notebook we want to write a comprehensive script that allows us to tackle inventory management strategy called FEFO, First Expiry First Out.

Let's start with the most basic situation, we have a dictionary of articles and quantities called "Inventory" and we want to iterate through it so to print the data it contains

In [6]:
# Create a variable called `inventory` and store some articles and quantities in it
inventory = {
    "apple": 100,
    "banana": 50,
    "orange": 75
}
# Write a loop to print the article and its quantity.
# We'll use the items() method to access the dictionary..
for k, v in inventory.items():
    print(k, v)


apple 100
banana 50
orange 75


Now we will build upon the `dictionary` object.
Tipically inventory data is stored in relational databases and its format is perfectly represented by that of a `list` of `dictionaries`. Each `dictionary` contains `keys` like 'article' or 'sku_code', 'quantity', 'description' and so on.

In [7]:
# Build a dictionary with three basic data points: 'article', 'quantity' and 'expiration_date'
inventory = [
    {"article": "apple", "quantity": 100, "expiration_date": "2025-01-15"},
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"},
    {"article": "orange", "quantity": 75, "expiration_date": "2025-01-20"}
]

# Learn to iterate through the dictionary accessing each entry or row and thus each `key - value` pair
for entry in inventory:
    print(f"{entry['article']}: {entry['quantity']}, expires on {entry['expiration_date']}")

apple: 100, expires on 2025-01-15
banana: 50, expires on 2025-01-10
orange: 75, expires on 2025-01-20


It is very important at this point that we step out of **Python** and think of the **logic** that we want to implement. Here it goes: 
1. we want to allocate existing inventory to a demand (we haven't yet dealt with demand, but we'll do so very soon) _in order of expiry date_ 
2. we want to dinamically return the remaining inventory after it has been allocated and consumed by production

Let's tackle 1. Ordering the inventory by expiration date. 
We will do so by using a lamda function to sort the inventory. We will also assign the sorted inventory to a new variable which will allow us to keep track of all steps of the way.

In [8]:
# Sort Inventory by Expiration Date
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

print("Sorted Inventory: \n")
for entry in sortinv:
    print(f"{entry['article']}: {entry['quantity']}, expires on {entry['expiration_date']}")

Sorted Inventory: 

banana: 50, expires on 2025-01-10
apple: 100, expires on 2025-01-15
orange: 75, expires on 2025-01-20


We will now build the demand dataset with a very simple list of two items. 
We will keep the same exact structure of the Inventory, i.e. a list of dictionaries, with keys 'article', 'quantity' and a date that tells us when the article is needed by production.

In [9]:
# Build the demand dataset
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-11"}
]

We will also want to order the demand by date, this is because we want to allocate the existing inventory to demand that is closer in time - for later demand lines we will have time to purchase more inventory!

In [10]:
# Sort demand by expiration date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])

print("Sorted Demand: \n")
for requir in sortdem:
    print(f"{requir['article']}: {requir['quantity']}, requirement on {requir['demand_date']}")

Sorted Demand: 

banana: 30, requirement on 2025-01-11
apple: 120, requirement on 2025-01-12


Now that we have both demand and inventory in a good shape, we want to tackle the second problem, i.e. allocating the inventory to the demand and returning the remaining datasets. 

As this is a dinamic and iterative process we will need to use a for loop. And this procedure will essentially do the following: 
- for each line of the demand, take the first line of the inventory with corresponding article (of course we fulfill demand with the correct parts)
- if the demand is larger than the inventory line, then subtract the inventory and reduce it to zero
- - thus search for any other lines in the inventory that can satisfy the remaining demand
- if the demand is less than the inventory, then make the demand zero (fulfill the demand) and store the remaining inventory 

For clarity purposes we will want to delete zero line quantities from the two datasets. 
Also, as before, for clarity purposes, we keep track of all steps of the way by deep-copying our datasets. 

In [11]:
import copy
calc_inventory = copy.deepcopy(sortinv)
calc_demand = copy.deepcopy(sortdem)

# Match demand with existing inventory
for d in calc_demand:
    for i in calc_inventory:
        if i['article'] == d['article']:
            if d['quantity'] >= i['quantity']:
                d['quantity'] -= i['quantity'] # demand - inventory
                i['quantity'] = 0 # thus save the remaining demand
            else:
                i['quantity'] -= d['quantity'] # inventory - demand
                d['quantity'] = 0 # then reduce the demand to zero
    calc_demand = [d for d in calc_demand if d['quantity'] > 0]
    calc_inventory = [i for i in calc_inventory if i['quantity'] > 0]
# Reproduce the remaining inventory
print("Remaining inventory:")
for i in calc_inventory:
    print(f"{i['article']}: {i['quantity']}")
# Reproduce the remaining demand
print("\nRemaining demand:")
for i in calc_demand:
    print(f"{i['article']}: {i['quantity']}")

Remaining inventory:
banana: 20
orange: 75

Remaining demand:
apple: 20


In [ ]:
# making a change